In [ ]:
import torch
import torch.nn.functional as F
import yaml
from types import SimpleNamespace

import sys
from pathlib import Path

# 1) figure out your project root (one level up from this notebook)
project_root = Path().resolve().parent

# 2) add it to the front of sys.path
sys.path.insert(0, str(project_root))

from src.models.moclip import build_moclip

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#Donwload MOCLIP
# Load YAML config file
with open('../src/models/config.yaml', 'r') as file:
    cfg_dict = yaml.safe_load(file)

# Convert to SimpleNamespace for easy attribute access
cfg = SimpleNamespace(**cfg_dict)

moclip = build_moclip(cfg)
moclip = moclip.to(device)

In [ ]:
# Get Geometry and Spectra encoders
geometry_encoder = moclip.geometry_encoder
spectra_encoder = moclip.spectra_encoder

In [ ]:
# Get test dataset
